# MCP tool discovery

## What
**Dynamic tool discovery** is a mechanism that allows an AI agent to discover available external tools without needing hardcoded knowledge of each one.
> Personal note: Like service discovery in microservices architecture, the AI agent queries the centralized MCP server (acts as a live catalog, or a "service registry").

This means the tools and the agent implementation are loosely coupled, and enable scalability.

## How
An MCP server hosts a set of functions that are exposed as tools using the `@mcp.tool` decorator. A client can connect to the server and fetch these tools dynamically. The client then generates function wrappers that are added to the Azure AI Agent's tool definitions.

- The MCP server hosts available tools.
- The MCP client dynamically discovers the tools.
- The Azure AI Agent uses the available tools to respond to user requests.

##  Why
This approach provides several benefits:
- Scalability: Easily add new tools or update existing ones without redeploying agents.
- Modularity: Agents can remain simple, focusing on delegation rather than managing tool details.
- Maintainability: Centralized tool management reduces duplication and errors.
- Flexibility: Supports diverse tool types and complex workflows by aggregating capabilities.

# MCP server and client
The MCP server hosts your tool catalog, and the MCP client fetches those tools and makes them usable by your agent.

## MCP Server
The MCP server can be initialized using FastMCP, which uses Python type hints and docstrings tro automatically generate tool definitions, which are served over HTTP.

## MCP Client
The MCP client acts as a bridge between your MCP server and the Azure AI Agent Service, it performs 3 key tasks:
- Discovers available tools from the MCP server using session.list_tools().
- Generates Python function stubs that wrap the tools.
- Registers those functions with your agent.

## Overview of MCP agent tool integration
- The MCP server hosts tool definitions decorated with `@mcp.tool`.
- The MCP client initializes an MCP client connection to the server.
- The MCP client fetches the available tool definitions with `session.list_tools()`.
- Each tool is wrapped in an async function that invokes `session.call_tool`
- The tool functions are bundled into `FunctionTool` that makes them usable by the agent.
- The `FunctionTool` is registered to the agent's toolset.

# Server setup

Refer to [aiagents_mcp_server.py](aiagents_mcp_server.py)

# Client setup

In [1]:
import os

from dotenv import load_dotenv

from mcp import ClientSession, StdioServerParameters
from mcp.client.stdio import stdio_client
from azure.ai.agents import AgentsClient
from azure.ai.agents.models import FunctionTool, MessageRole, ListSortOrder
from azure.identity import DefaultAzureCredential
from contextlib import AsyncExitStack

load_dotenv()

AGENT_PROJECT_ENDPOINT = os.getenv("AGENT_PROJECT_ENDPOINT")
AGENT_MODEL_DEPLOYMENT = os.getenv("AGENT_MODEL_DEPLOYMENT")

In [2]:
async def connect_to_server(exit_stack: AsyncExitStack):
    server_params = StdioServerParameters(
        command="python",
        args=["aiagents_mcp_server.py"],
        env=None
    )

    # Start the MCP server
    stdio_transport = await exit_stack.enter_async_context(stdio_client(server_params))
    stdio, write = stdio_transport

    # Create an MCP client session
    session = await exit_stack.enter_async_context(ClientSession(stdio, write))
    await session.initialize()

    # List available tools
    response = await session.list_tools()
    tools = response.tools
    print("\nConnected to server with tools:", [tool.name for tool in tools]) 
    
    return session

In [3]:
exit_stack = AsyncExitStack()
session = await connect_to_server(exit_stack)


Connected to server with tools: ['get_inventory_levels', 'get_weekly_sales']


In [22]:

response = await session.list_tools()
response.tools[0]

Tool(name='get_inventory_levels', title=None, description='Returns the current inventory levels for all products.\n\nReturns:\n    dict: A dictionary where the keys are product names (str)\n          and the values are inventory counts (int).', inputSchema={'properties': {}, 'type': 'object'}, outputSchema={'additionalProperties': True, 'type': 'object'}, annotations=None, meta=None)

In [36]:
# Initialize the agent client
agents_client = AgentsClient(
    endpoint=AGENT_PROJECT_ENDPOINT,
    credential=DefaultAzureCredential(
        exclude_environment_credential=True,
        exclude_managed_identity_credential=True
    )
)

response = await session.list_tools()
tools = response.tools

# Build a function for each tool
def make_tool_func(tool_name, tool_desc):
     async def tool_func(**kwargs):
         result = await session.call_tool(tool_name, kwargs)
         return result
        
     tool_func.__name__ = tool_name
     tool_func.__doc__ = tool_desc
     return tool_func

functions_dict = {tool.name: make_tool_func(tool.name, tool.description) for tool in tools}
mcp_function_tool = FunctionTool(functions=list(functions_dict.values()))

# Workaround to remove the required kwargs condition
for i in range(len(mcp_function_tool.definitions)):
    mcp_function_tool.definitions[i]["function"]["parameters"] = {}

mcp_function_tool.definitions

[{'type': 'function', 'function': {'name': 'get_inventory_levels', 'description': 'Returns the current inventory levels for all products.', 'parameters': {}}},
 {'type': 'function', 'function': {'name': 'get_weekly_sales', 'description': 'Returns the number of units sold for each product in the past week.', 'parameters': {}}}]

In [37]:
# Create the agent
agent = agents_client.create_agent(
     model=AGENT_MODEL_DEPLOYMENT,
     name="inventory-agent",
     instructions="""
     You are an inventory assistant. Here are some general guidelines:
     - Recommend restock if item inventory < 10  and weekly sales > 15
     - Recommend clearance if item inventory > 20 and weekly sales < 5
     """,
     tools=mcp_function_tool.definitions
)

# Enable auto function calling
agents_client.enable_auto_function_calls(tools=mcp_function_tool)

In [38]:
agent.tools

[{'type': 'function', 'function': {'name': 'get_inventory_levels', 'description': 'Returns the current inventory levels for all products.', 'parameters': {'type': 'object', 'properties': {}}, 'strict': False}},
 {'type': 'function', 'function': {'name': 'get_weekly_sales', 'description': 'Returns the number of units sold for each product in the past week.', 'parameters': {'type': 'object', 'properties': {}}, 'strict': False}}]

In [39]:
# Create a thread for the chat session
thread = agents_client.threads.create()

In [40]:
import time
import json


async def send_message(user_prompt: str):
    # Invoke the prompt
    message = agents_client.messages.create(
        thread_id=thread.id,
        role=MessageRole.USER,
        content=user_prompt,
    )
    run = agents_client.runs.create(thread_id=thread.id, agent_id=agent.id)

    # Monitor the run status
    while run.status in ["queued", "in_progress", "requires_action"]:
        time.sleep(1)
        run = agents_client.runs.get(thread_id=thread.id, run_id=run.id)
        tool_outputs = []

        if run.status == "requires_action":
            tool_calls = run.required_action.submit_tool_outputs.tool_calls
            print(f"[Tool calls]: {tool_calls}")
            for tool_call in tool_calls:
                # Retrieve the matching function tool
                function_name = tool_call.function.name
                args_json = tool_call.function.arguments
                kwargs = json.loads(args_json)
                required_function = functions_dict.get(function_name)
                output = await required_function(**kwargs)

                # Append the output text
                tool_outputs.append({
                    "tool_call_id": tool_call.id,
                    "output": output.content[0].text,
                })
            
            # Submit the tool call output
            agents_client.runs.submit_tool_outputs(thread_id=thread.id, run_id=run.id, tool_outputs=tool_outputs)
            
    # Check for failure
    if run.status == "failed":
        print(f"Run failed: {run.last_error}")

    # Display the response
    messages = agents_client.messages.list(thread_id=thread.id, order=ListSortOrder.ASCENDING)
    for message in messages:
        if message.text_messages:
            last_msg = message.text_messages[-1]
            print(f"{message.role}:\n{last_msg.text.value}\n")

In [41]:
await send_message("What are the current inventory levels?")

[Tool calls]: [{'id': 'call_WUWoYannDSzSgzp4VclNdu1N', 'type': 'function', 'function': {'name': 'get_inventory_levels', 'arguments': '{}'}}]
MessageRole.USER:
What are the current inventory levels?

MessageRole.AGENT:
Here are the current inventory levels for each product:

- **Moisturizer**: 6
- **Shampoo**: 8
- **Body Spray**: 28
- **Hair Gel**: 5
- **Lip Balm**: 12
- **Skin Serum**: 9
- **Cleanser**: 30
- **Conditioner**: 3
- **Setting Powder**: 17
- **Dry Shampoo**: 45



In [42]:
await send_message("What is the number of shampoos sold last week?")

[Tool calls]: [{'id': 'call_Xu5hfl3NWt6Viq7PcEAUVQh9', 'type': 'function', 'function': {'name': 'get_weekly_sales', 'arguments': '{}'}}]
MessageRole.USER:
What are the current inventory levels?

MessageRole.AGENT:
Here are the current inventory levels for each product:

- **Moisturizer**: 6
- **Shampoo**: 8
- **Body Spray**: 28
- **Hair Gel**: 5
- **Lip Balm**: 12
- **Skin Serum**: 9
- **Cleanser**: 30
- **Conditioner**: 3
- **Setting Powder**: 17
- **Dry Shampoo**: 45

MessageRole.USER:
What is the number of shampoos sold last week?

MessageRole.AGENT:
Last week, **18 shampoos** were sold.



# Clean up

In [43]:
agents_client.delete_agent(agent.id)

In [ ]:
await exit_stack.aclose()